In [20]:
import numpy as np
import trimesh
import networkx as nx
from sklearn.metrics.pairwise import haversine_distances
from sklearn.neighbors import BallTree
from scipy.spatial.transform import Rotation as R

In [ ]:
def direction_vec(v1, v2, epsilon=10e-11):
    v = np.cross(v1, v2)
    vnorm1 = np.dot(v, v)
    if vnorm1 < epsilon:
        v1 = v1 + epsilon
        v = np.cross(v1, v2)
        vnorm1 = np.dot(v, v)
    return v / np.sqrt(vnorm1)

In [1]:
def create_sphere(subdivisions=0, radius=1.0):
    return trimesh.creation.icosphere(subdivisions=subdivisions, radius=radius)

In [18]:
def to_unit_sphere_xyz(loc):
    latr, lonr = loc
    R = 1.0
    x = R * np.cos(latr) * np.cos(lonr)
    y = R * np.cos(latr) * np.sin(lonr)
    z = R * np.sin(latr)
    return np.array((x, y, z))

In [19]:
def get_rotation_from_unit_vecs(v1, v2):
    v_unit = direction_vec(v1, v2)
    theta = np.arccos(np.dot(v1, v2))
    return R.from_rotvec(v_unit * theta)

In [4]:
def to_latlon(xyz, radius=1.0):
    lat = np.arcsin(xyz[..., 2] / radius) * 180.0 / np.pi
    lon = np.arctan2(xyz[..., 1], xyz[..., 0]) * 180.0 / np.pi
    return np.array((lat, lon), dtype=np.float32).transpose()

In [7]:
def to_rad(xyz, radius=1.0):
    lat = np.arcsin(xyz[..., 2] / radius)
    lon = np.arctan2(xyz[..., 1], xyz[..., 0])
    return np.array((lat, lon), dtype=np.float32).transpose()

In [17]:
def compute_directions(loc1, loc2, pole_vec=(0.0, 0.0, 1.0)):
    pole_vec = np.array(pole_vec)  # all will be rotated relative to destination node
    loc1_xyz = to_unit_sphere_xyz(loc1)
    loc2_xyz = to_unit_sphere_xyz(loc2)
    r = get_rotation_from_unit_vecs(loc2_xyz, pole_vec)  # r.apply(loc1_xyz) == pole_vec
    direction = direction_vec(r.apply(loc1_xyz), pole_vec)
    return direction / np.sqrt(np.dot(direction, direction))

In [16]:
def directional_edge_features_rotated(loc1, loc2):
    return compute_directions(loc1, loc2)[0:2]  # discard last component -> zero if rotated to north pole

In [8]:
def get_one_ring(sp):
    g = nx.from_edgelist(sp.edges_unique)
    one_ring = [list(g[i].keys()) for i in range(len(sp.vertices))]

    return one_ring

In [9]:
def add_edge(G, idx1, idx2, allow_self_loop=False, add_edge_attrib=False):
    loc1 = np.deg2rad(h3.h3_to_geo(idx1))
    loc2 = np.deg2rad(h3.h3_to_geo(idx2))
    if allow_self_loop or idx1 != idx2:
        if add_edge_attrib:
            direction = directional_edge_features_rotated(loc1, loc2)
            G.add_edge(
                idx1,
                idx2,
                edge_attr=(haversine_distances([loc1, loc2])[0][1], *direction),
            )

        else:
            G.add_edge(idx1, idx2, weight=haversine_distances([loc1, loc2])[0][1])
            # G.add_edge(idx1, idx2, weight = h3.point_dist(loc1, loc2, unit='rads'))


In [10]:
def add_nodes(G, resolution, self_loop=False):
    for idx in h3.uncompact(h3.get_res0_indexes(), resolution):
        G.add_node(idx, hcoords_rad=np.deg2rad(h3.h3_to_geo(idx)))
        if self_loop:
            add_edge(G, idx, idx, allow_self_loop=self_loop)

In [6]:
def multi_mesh2(resolutions):
    G = nx.Graph()

    sp1 = create_sphere(resolutions[-1])
    sp1_rad = to_rad(sp1.vertices)
    one_rings = get_one_ring(sp1)

    for ii, coords in enumerate(sp1_rad):
        G.add_node(ii, hcoords_rad=sp1_rad[ii])

    for ii in range(len(sp1.vertices)):
        for ineighb in one_rings[ii]:
            if ineighb != ii:
                loc_self = sp1_rad[ii]
                loc_neigh = sp1_rad[ineighb]
                # direction = directional_edge_features_rotated(loc_neigh, loc_self)
                # G.add_edge(ineighb, ii, edge_attr=(haversine_distances([loc_neigh, loc_self])[0][1], *direction))
                G.add_edge(ineighb, ii, weight=haversine_distances([loc_neigh, loc_self])[0][1])

    tree = BallTree(sp1_rad, metric="haversine")

    for resolution in resolutions[:-1]:
        sp2 = create_sphere(resolution)
        sp2_rad = to_rad(sp2.vertices)
        one_rings = get_one_ring(sp2)

        dist, ind1 = tree.query(sp2_rad, k=1)
        for ii in range(len(sp2.vertices)):
            for ineighb in one_rings[ii]:
                if ineighb != ii:
                    loc_dst = sp2_rad[ii]
                    loc_neigh = sp2_rad[ineighb]
                    # direction = directional_edge_features_rotated(loc_neigh, loc_dst)
                    # G.add_edge(ind1[ineighb][0], ind1[ii][0], \
                    # edge_attr=(haversine_distances([loc_neigh, loc_dst])[0][1], *direction))
                    G.add_edge(ind1[ineighb][0], ind1[ii][0], weight=haversine_distances([loc_neigh, loc_dst])[0][1])

    return G, sp1_rad